# 使用 Milvus 和 DeepSeek 构建民法典 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

该文档使用 Milvus 和 DeepSeek 构建一个基于民法典（节选）文档的检索增强生成 (RAG) 管道。


## 准备工作


### 依赖与环境


In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0


---


In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")


### 准备数据


我们使用民法典（节选）文档 `mfd.md` 作为我们 RAG 中的私有知识库。该文档包含了物权编和合同编的内容。


我们从 `mfd.md` 文件加载内容。对于文档，我们使用 "**" 来分割法条，这样可以大致分离出每个法条的内容。


In [3]:
import re

# 读取民法典文档
with open("mfd.md", "r", encoding="utf-8") as file:
    file_text = file.read()

# 使用正则表达式分割法条，保留法条编号和内容
# 匹配模式：**第XXX条** 或 **第XXX条** 开头的内容
text_lines = []
current_section = ""

# 按行处理，将相关法条组合在一起
lines = file_text.split("\n")
current_chunk = []

for line in lines:
    # 如果遇到法条编号（**第XXX条**），保存之前的chunk并开始新的
    if re.match(r'\*\*第[零一二三四五六七八九十百千万]+条\*\*', line) or re.match(r'\*\*第\d+条\*\*', line):
        if current_chunk:
            text_lines.append("\n".join(current_chunk))
        current_chunk = [line]
    elif line.strip() and not line.startswith("#"):
        # 非空行且不是标题，添加到当前chunk
        current_chunk.append(line)
    elif line.startswith("#"):
        # 遇到标题，保存之前的chunk
        if current_chunk:
            text_lines.append("\n".join(current_chunk))
        current_chunk = []
        # 标题也作为一个chunk
        text_lines.append(line)

# 保存最后一个chunk
if current_chunk:
    text_lines.append("\n".join(current_chunk))

# 过滤掉空内容
text_lines = [line.strip() for line in text_lines if line.strip()]

print(f"总共分割出 {len(text_lines)} 个文本块")


总共分割出 416 个文本块


In [5]:
# 查看前几个文本块的内容
for i, line in enumerate(text_lines[:30]):
    print(f"\n=== 文本块 {i+1} ===")
    print(line[:200] + "..." if len(line) > 200 else line)



=== 文本块 1 ===
## 中华人民共和国民法典

=== 文本块 2 ===
### （二）物权编

=== 文本块 3 ===
#### 第一章 一般规定

=== 文本块 4 ===
**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。

=== 文本块 5 ===
**第二百零五条** 本编调整因物的归属和利用产生的民事关系。

=== 文本块 6 ===
**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。
国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。
国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。

=== 文本块 7 ===
**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。

=== 文本块 8 ===
**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。

=== 文本块 9 ===
**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。
依法属于国家所有的自然资源，所有权可以不登记。

=== 文本块 10 ===
**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。
国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。

=== 文本块 11 ===
**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。
申请登记材料以及登记事项相关信息，可以公开查询。

=== 文本块 12 ===
**第二百一十二条** 登记机构应当履行下列职责：
（一）审查申请人提供的材料；
（二）询问申请人；
（三）如实、及时登记；
（四）法律、行政法规规定的其他职责。
申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。

=== 文本块 13 ===
**第二百一十三条** 登记机构不得有下列行为：
（一）要求对不动产进行评估；
（二）以不动产登记为条件收取其他费用；
（三）超出登记职责范围的其他行为。

=== 文本块 14 ==

### 准备 LLM 和 Embedding 模型


DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。


In [6]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)


定义一个 embedding 模型，使用 OpenAI 的 embedding 模型来生成文本嵌入。


In [7]:
from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-6yR6RvofW3KRAOsp16B03e947cCb41C5A778Ee29Be2dFd32', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)


/Users/wangyaxin/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


生成一个测试嵌入并打印其维度和前几个元素。


In [8]:
test_embedding = embedding_model.encode_queries(["这是一个测试"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])


512
[ 3.54619720e-03 -2.36984288e-05  1.52892778e-02  8.20477232e-02
  2.84169130e-02 -1.78138223e-02 -4.73667793e-02  1.58415213e-01
  2.13955212e-02 -1.59361921e-02]


## 将数据加载到 Milvus


### 创建 Collection


In [9]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_civil_code.db")

collection_name = "civil_code_rag_collection"


/Users/wangyaxin/miniconda3/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


检查 collection 是否已存在，如果存在则删除它。


In [10]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)


创建一个具有指定参数的新 collection。


In [11]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 强一致性
)


### 插入数据


遍历文本块，创建嵌入，然后将数据插入 Milvus。


In [12]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)


Creating embeddings: 100%|███████████████████| 416/416 [00:00<00:00, 1172601.12it/s]


{'insert_count': 416, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

## 构建 RAG


### 问题 1：关于不动产登记


In [13]:
question1 = "不动产物权的设立、变更、转让和消灭需要登记吗？登记后什么时候生效？"


在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。


In [14]:
search_res1 = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries([question1]),
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)


In [16]:
import json

retrieved_lines_with_distances1 = [
    (res["entity"]["text"], res["distance"]) for res in search_res1[0]
]
print("检索到的相关内容：")
print(json.dumps(retrieved_lines_with_distances1, indent=4, ensure_ascii=False))


检索到的相关内容：
[
    [
        "**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。",
        0.7497363090515137
    ],
    [
        "**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。",
        0.7205978631973267
    ],
    [
        "**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。",
        0.7081194519996643
    ]
]


将检索到的文档转换为字符串格式，并使用 LLM 生成回答。


In [18]:
context1 = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances1]
)

SYSTEM_PROMPT = """
Human: 你是一个专业的法律AI助手。你能够从提供的民法典条文片段中找到问题的准确答案。
"""
USER_PROMPT1 = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context1}
</context>
<question>
{question1}
</question>
"""

response1 = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT1},
    ],
)
print("=" * 80)
print("问题 1：")
print(question1)
print("\n回答：")
print(response1.choices[0].message.content)
print("=" * 80)


问题 1：
不动产物权的设立、变更、转让和消灭需要登记吗？登记后什么时候生效？

回答：
根据《中华人民共和国民法典》的相关规定：

1. **不动产物权的设立、变更、转让和消灭需要登记吗？**  
   需要。根据第二百零九条，不动产物权的设立、变更、转让和消灭，经依法登记发生效力；未经登记，不发生效力，但法律另有规定的除外。此外，依法属于国家所有的自然资源，所有权可以不登记。

2. **登记后什么时候生效？**  
   根据第二百一十四条，不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。

因此，不动产物权的变动原则上需要依法登记，且自记载于不动产登记簿时生效。


### 问题 2：关于合同订立


In [19]:
question2 = "合同订立包括哪些阶段？要约和承诺分别是什么？"


In [20]:
search_res2 = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries([question2]),
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

retrieved_lines_with_distances2 = [
    (res["entity"]["text"], res["distance"]) for res in search_res2[0]
]
print("检索到的相关内容：")
print(json.dumps(retrieved_lines_with_distances2, indent=4, ensure_ascii=False))


检索到的相关内容：
[
    [
        "**第四百八十七条** 合同的订立包括要约和承诺阶段。",
        0.7212382555007935
    ],
    [
        "**第四百八十八条** 要约是希望与他人订立合同的意思表示，该意思表示应当符合下列规定：\n（一）内容具体确定；\n（二）表明经受要约人承诺，要约人即受该意思表示约束。",
        0.5761759877204895
    ],
    [
        "**第四百九十四条** 承诺是对要约的同意。\n承诺应当以通知的方式作出，但是根据交易习惯或者要约表明可以通过行为作出承诺的除外。",
        0.5521177053451538
    ]
]


In [21]:
context2 = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances2]
)

USER_PROMPT2 = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context2}
</context>
<question>
{question2}
</question>
"""

response2 = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT2},
    ],
)
print("=" * 80)
print("问题 2：")
print(question2)
print("\n回答：")
print(response2.choices[0].message.content)
print("=" * 80)


问题 2：
合同订立包括哪些阶段？要约和承诺分别是什么？

回答：
根据提供的民法典条文片段，合同订立包括要约和承诺阶段。要约是希望与他人订立合同的意思表示，其内容需具体确定，并表明经受要约人承诺，要约人即受该意思表示约束。承诺是对要约的同意，通常应以通知方式作出，但根据交易习惯或要约表明可以通过行为作出承诺的除外。
